# 좌표로 행정동 가져오기

## 버스 정류장

In [1]:
import pandas as pd
import numpy as np
import requests

In [14]:
bus_stop = pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='CP949', names = ['node_id', 'ars_id', 'busstop_name', 'latitude', 'longitude' ], skiprows=1)
bus_stop.head()

,node_id,ars_id,busstop_name,latitude,longitude
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230


In [3]:
def juso(x) :
    method = "GET"
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    params = {'x' : x['latitude'], 'y' : x['longitude']}
    header = {'authorization': 'KakaoAK apikey', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    
    try :
        response = requests.request(method=method, url=url, headers=header, params=params )
        tokens = response.json()
        x['gu_name'] = tokens['documents'][1]['region_2depth_name']
        x['dong_name'] = tokens['documents'][1]['region_3depth_name']
        x['full_name'] = tokens['documents'][1]['region_1depth_name']+' '+tokens['documents'][1]['region_2depth_name']+' '+tokens['documents'][1]['region_3depth_name']
    except :
        x['gu_name'] = ''
        x['dong_name'] = ''
        x['full_name'] = ''
    return x

In [4]:
bus_stop = bus_stop.apply(juso, axis =1).drop(['node_id','ars_id'], axis=1)

In [5]:
bus_stop.head()

,busstop_name,latitude,longitude,gu_name,dong_name,full_name
0,정류소명,좌표X,좌표Y,,,
1,종로2가사거리,126.9877861534,37.5697641508,종로구,종로1.2.3.4가동,서울특별시 종로구 종로1.2.3.4가동
2,창경궁.서울대학교병원,126.9965201597,37.5791788583,종로구,종로1.2.3.4가동,서울특별시 종로구 종로1.2.3.4가동
3,명륜3가.성대입구,126.9982901625,37.5827088189,종로구,혜화동,서울특별시 종로구 혜화동
4,종로2가.삼일교,126.9875072018,37.5685822333,종로구,종로1.2.3.4가동,서울특별시 종로구 종로1.2.3.4가동


In [6]:
bus_stop = bus_stop[bus_stop['full_name'].str.contains('서울특별시')]

In [7]:
bus_stop.to_csv('버스 정류장 주소.csv', index=False, encoding='CP949')

## 지하철역

In [8]:
subway = pd.read_csv('station_coordinate.csv', names = ['line', 'station_name', 'ID', 'longitude', 'latitude'], skiprows=1, encoding='CP949')
subway.head()

,line,station_name,ID,longitude,latitude
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [9]:
subway = subway.apply(juso, axis =1).drop(['ID'], axis=1)

In [10]:
subway = subway[subway['full_name'].str.contains('서울특별시')]

In [11]:
subway.head()

,line,station_name,longitude,latitude,gu_name,dong_name,full_name
1,01호선,남영,37.541021,126.971300,용산구,남영동,서울특별시 용산구 남영동
2,01호선,용산,37.529849,126.964561,용산구,한강로동,서울특별시 용산구 한강로동
3,01호선,노량진,37.514219,126.942454,동작구,노량진1동,서울특별시 동작구 노량진1동
4,01호선,대방,37.513342,126.926382,영등포구,신길7동,서울특별시 영등포구 신길7동
5,01호선,영등포,37.515504,126.907628,영등포구,영등포본동,서울특별시 영등포구 영등포본동


In [12]:
subway.to_csv('지하철역 주소.csv', index=False, encoding='CP949')